# Creating Workflows

Learn how to build multi-step workflows using Victor's StateGraph engine and YAML workflow compiler.

## Learning Objectives

By the end of this notebook, you'll:
- Understand workflow concepts
- Create YAML workflows
- Build StateGraph workflows in Python
- Use conditional routing
- Implement human-in-the-loop approvals
- Run parallel workflows

## 1. What is a Workflow?

A **workflow** is a multi-step process that:
- Defines multiple stages of execution
- Can include conditional logic
- Supports parallel execution
- Maintains state across steps
- Can include human approvals

Victor supports two approaches:
1. **YAML Workflows** - Declarative and easy to read
2. **StateGraph** - Programmatic with full flexibility

## 2. YAML Workflow Example

Let's create a simple workflow in YAML:

In [ ]:
# First, let's create a simple YAML workflow
yaml_workflow = """
name: "Content Summarizer"
description: "Analyze and summarize content"

nodes:
  - id: "analyze"
    type: "agent"
    config:
      prompt: |
        Analyze this content: {{input}}
        Identify the main topics and key points.

  - id: "summarize"
    type: "agent"
    config:
      prompt: |
        Create a 2-sentence summary based on:
        {{analyze.output}}

edges:
  - from: "start"
    to: "analyze"
  - from: "analyze"
    to: "summarize"
  - from: "summarize"
    to: "complete"
"""

# Save it
import tempfile
import os

with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(yaml_workflow)
    workflow_path = f.name

print(f"Workflow saved to: {workflow_path}")

In [ ]:
# Now let's run it
import asyncio
from victor import Agent

async def run_workflow():
    agent = Agent.create()
    
    result = await agent.run_workflow(
        workflow_path,
        input={"input": "Victor is an AI framework for building agents."}
    )
    
    return result

# Run the workflow
result = asyncio.run(run_workflow())
print(result.content)

# Cleanup
os.unlink(workflow_path)

## 3. StateGraph in Python

For more complex workflows, use StateGraph:

In [ ]:
from victor.framework import StateGraph

# Define workflow nodes
async def analyze_node(state):
    """Analyze the input."""
    prompt = f"Analyze: {state['input']}"
    agent = Agent.create()
    result = await agent.run(prompt)
    return {"analysis": result.content}

async def summarize_node(state):
    """Summarize the analysis."""
    prompt = f"Summarize: {state['analysis']}"
    agent = Agent.create()
    result = await agent.run(prompt)
    return {"summary": result.content}

# Build the workflow
workflow = StateGraph()

# Add nodes
workflow.add_node("analyze", analyze_node)
workflow.add_node("summarize", summarize_node)

# Add edges
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "summarize")
workflow.set_finish_point("summarize")

print("Workflow created!")

In [ ]:
# Compile and run the workflow
compiled = workflow.compile()

result = await compiled.ainvoke({"input": "Victor is an AI framework for building agents."})

print(f"Analysis: {result['analysis']}")
print(f"\nSummary: {result['summary']}")

## 4. Conditional Routing

Route workflows based on conditions:

In [ ]:
async def classify_node(state):
    """Classify input as technical or general."""
    agent = Agent.create()
    result = await agent.run(
        f"Classify as [technical, general]: {state['input']}"
    )
    return {"category": result.content.strip().lower()}

async def technical_handler(state):
    """Handle technical content."""
    return {"result": "Technical analysis complete"}

async def general_handler(state):
    """Handle general content."""
    return {"result": "General analysis complete"}

def route_function(state):
    """Route based on classification."""
    category = state.get("category", "general")
    return "technical" if "technical" in category else "general"

# Build workflow
workflow = StateGraph()
workflow.add_node("classify", classify_node)
workflow.add_node("technical", technical_handler)
workflow.add_node("general", general_handler)

workflow.set_entry_point("classify")

# Add conditional routing
workflow.add_conditional_edges(
    "classify",
    route_function,
    {
        "technical": "technical",
        "general": "general"
    }
)

workflow.set_finish_point("technical")
workflow.set_finish_point("general")

# Test with technical input
compiled = workflow.compile()
result = await compiled.ainvoke({"input": "How does async/await work in Python?"})
print(result)

## 5. Parallel Execution

Run multiple steps in parallel:

In [ ]:
async def research_agent(state, perspective):
    """Research from a specific perspective."""
    agent = Agent.create()
    result = await agent.run(
        f"Research '{state['topic']}' from a {perspective} perspective."
    )
    return {"result": result.content}

async def technical_research(state):
    return await research_agent(state, "technical")

async def business_research(state):
    return await research_agent(state, "business")

async def synthesize(state):
    """Combine research results."""
    agent = Agent.create()
    result = await agent.run(
        f"Synthesize these perspectives:\n"
        f"Technical: {state.get('technical', 'N/A')}\n\n"
        f"Business: {state.get('business', 'N/A')}"
    )
    return {"synthesis": result.content}

# Build parallel workflow
workflow = StateGraph()
workflow.add_node("technical", technical_research)
workflow.add_node("business", business_research)
workflow.add_node("synthesize", synthesize)

# Both research nodes start from 'start'
# Both feed into synthesize
# Note: In actual usage, you'd use workflow.set_entry_point()
# and proper edge configuration

print("Parallel workflow structure created!")

## 6. Streaming Workflow Execution

Monitor workflow progress in real-time:

In [ ]:
from victor import Agent

async def stream_workflow():
    agent = Agent.create()
    
    # Stream workflow execution
    print("Workflow execution:\n")
    
    async for node_id, state in agent.stream_workflow(
        workflow_path,
        input={"input": "Test content"}
    ):
        print(f"✓ Completed: {node_id}")
        if "output" in state:
            print(f"  Output: {state['output'][:50]}...")

# Note: This would use the workflow_path from earlier
# For this example, we'll just show the pattern
print("Streaming pattern demonstrated!")

## Exercise: Build Your Workflow

Create a workflow that:
1. Takes a user question
2. Researches the answer
3. Summarizes findings
4. Provides a final answer

💡 Use the cell below:

In [ ]:
# Your workflow here


# Test it
# result = await workflow.invoke({"question": "What is AI?"})

## Summary

In this notebook, you learned:
- ✅ Creating YAML workflows
- ✅ Building StateGraph workflows
- ✅ Conditional routing
- ✅ Parallel execution
- ✅ Streaming workflow progress

## Next Steps

Continue to [03_tools.ipynb](./03_tools.ipynb) to learn about using tools!